In [1]:
## standard library
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import pandas as pd
import time
import json
import pickle
import os

#Image processing packages
import cv2
from PIL import Image

## Keras + Tensorflow
from keras.models import load_model
from keras.models import Sequential
from keras.utils import np_utils
import tensorflow as tf

from sklearn.utils import shuffle

%matplotlib inline

Using TensorFlow backend.


## Used for conducting test on full satellite image. No training of model. Model is manually pick and run in this notebook

Description of the below function are found in Capstone Part 4.ipynb

Slight modification to functions written in Capstone Part 4.ipynb

Code here is keep short for easy transfer to a python script.

In [2]:
def sliding_window(image, stepsize , winsize):
    for y in range(0, image.shape[0], stepsize[0]):
        for x in range(0, image.shape[1], stepsize[1]):
            yield (x, y, x + winsize[1], y + winsize[0], image[y:y + winsize[0], x:x + winsize[1]])

In [3]:
def non_max_suppression(bound_box, area_overlap_threshold):
    boxes = np.array(bound_box)
    if len(boxes) == 0:
        return []
    pick = []
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    area = 6400.0
    ind = np.argsort(y2)
    
    while len(ind) > 0:
        end = len(ind) - 1
        i = ind[end] 
        pick.append(i) 
        suppress = [end]
        
        for coord in range(0, end):
            j = ind[coord]
            xx1 = max(x1[i], x1[j])
            yy1 = max(y1[i], y1[j])
            xx2 = min(x2[i], x2[j])
            yy2 = min(y2[i], y2[j])
        
            w = max(0, xx2 - xx1 + 1)
            h = max(0, yy2 - yy1 + 1)

            overlap = (w * h) / area
        
            if overlap > area_overlap_threshold:
                suppress.append(coord)

        ind = np.delete(ind, suppress)
 
    return boxes[pick]

In [10]:
def scan_image(satimage_in, satimage_out, model, dir_window, dir_gif, dir_results, winH=80, winW=80, stepY=10, stepX=10):
    
    ## change to a function for scan image instead of just code to allow quick change of model
    
    ship_pred_class = [] 
    ship_img = [] 
    ship_cooridinate = [] 

    i = 0
    
    print('-------------------')
    print('Scan image start')
    print('-------------------')
    
    scan_start = time.time()
    
    for (x1, y1, x2, y2, window) in sliding_window(satimage_in, stepsize=(stepY,stepX), winsize=(winH, winW)):
    
        if window.shape[0] != winH or window.shape[1] != winW:
            continue

        target_region = np.expand_dims(window, axis=0)
        classes = model.predict_classes(target_region, batch_size=1, verbose=0)
    
        if classes[0] == 1:
            
            #Don't save window image classified as ship
            
            #cv2.imwrite(dir_window + "rimg_{}.png".format(i), window)
        
            ship_pred_class.append(classes[0])
            ship_img.append(window)
            ship_cooridinate.append((x1,y1,x2,y2))
        
            cv2.rectangle(satimage_out, (x1, y1), (x1 + winW, y1 + winH), (0, 255, 0), 2) 
            
            #Don't save window image classified as full sat image for gif
            
            #cv2.imwrite(dir_gif + "img_{}.png".format(i), satimage_out)
                
            #i += 1
            
            # This is for debugging purpose to see visually if the code is working properly
        
            #cv2.waitKey(1)
            #time.sleep(0.025)
        #else:
            #cv2.waitKey(1)
            #time.sleep(0.025)

    print('Save window image (classified as ship) and gif image')
    
    
    ## Change write output directory for each test case
    
    cv2.imwrite(dir_results + "detectedtest4.png", satimage_out)
    cv2.destroyAllWindows()
                          
    scan_end = time.time()
    scan_time = scan_end - scan_start

    print("scan time takes = {} s".format(scan_time))

    print('-------------------')
    
    return ship_pred_class, ship_img, ship_cooridinate

## 4 test images
    1) Original sfbat_1.png (sometime taken around September according to Kaggle competition starter)
    2) Same location in SFbay but on 2017-11-01
    3) Long Beach location on 2017-10-18
    4) Long Beach location on 2017-10-17 

Purpose of this script is to collect final satellite image results with bounding box drawn on it.

Refresh Kernel for each run of images to prevent plotting over the same image 

(opencv2 does not refresh memory for cv2.imwrite if run from jupyter notebook)

## 1) Original Test Set (sometime around September)

In [ ]:
satimage_in = cv2.imread('./ships-in-satellite-imagery/test_v2/sfbay_1.png')
satimage_out = cv2.imread('./ships-in-satellite-imagery/test_v2/sfbay_1.png')
satimage_nms = cv2.imread('./ships-in-satellite-imagery/test_v2/sfbay_1.png')


model = load_model('./cnn_model/classiferCNN/model11.h5')

(winH, winW) = (80, 80)
(stepY,stepX) = (10, 10)

dir_window = './results/capstoneP5/winimg1/'
dir_gif = './results/capstoneP5/gif1/'
dir_results = './results/capstoneP5/final_result/'

print('---------------------------')
print('Scan Sat Image')
print('---------------------------')

ship_pred_class, ship_img, ship_cooridinate = scan_image(satimage_in, satimage_out, model, dir_window, dir_gif, dir_results)

print('---------------------------')
print('NMS')
print('---------------------------')

picked_box = non_max_suppression(ship_cooridinate, area_overlap_threshold = 0.5)

for (bbx1, bby1, bbx2, bby2) in picked_box:
    cv2.rectangle(satimage_nms, (bbx1, bby1), (bbx2,bby2), (0, 0, 255), 2) 

        
cv2.imwrite("./results/capstoneP5/final_result/detected11_NMS.png", satimage_nms)

print('Save NMS results')


In [5]:
## Save data of images classified as ship

In [ ]:
predict_ship1 = {'ship_pred_class': ship_pred_class, 'ship_img': ship_img, 'ship_cooridinate': ship_cooridinate}
    
with open('./results/capstoneP5/final_result/predict_ship11.pkl', 'wb') as file_name:
    pickle.dump(predict_ship1, file_name)

## 2) TEST IMAGE ON 2017-11-01 SF Bay

In [5]:
satimage_in = cv2.imread('./ships-in-satellite-imagery/test_v2/20171101.png')
satimage_out = cv2.imread('./ships-in-satellite-imagery/test_v2/20171101.png')
satimage_nms = cv2.imread('./ships-in-satellite-imagery/test_v2/20171101.png')

## Model 9 best model to predict another time

model = load_model('./cnn_model/classiferCNN/model9.h5')

(winH, winW) = (80, 80)
(stepY,stepX) = (10, 10)

dir_window = './results/capstoneP5/winimg1/'
dir_gif = './results/capstoneP5/gif1/'
dir_results = './results/capstoneP5/final_result/'

print('---------------------------')
print('Scan Sat Image')
print('---------------------------')

ship_pred_class, ship_img, ship_cooridinate = scan_image(satimage_in, satimage_out, model, dir_window, dir_gif, dir_results)

print('---------------------------')
print('NMS')
print('---------------------------')

picked_box = non_max_suppression(ship_cooridinate, area_overlap_threshold = 0.5)

for (bbx1, bby1, bbx2, bby2) in picked_box:
    cv2.rectangle(satimage_nms, (bbx1, bby1), (bbx2,bby2), (0, 0, 255), 2) 

        
cv2.imwrite("./results/capstoneP5/final_result/detectedtest2_NMS.png", satimage_nms)

print('Save NMS results')

---------------------------
Scan Sat Image
---------------------------
-------------------
Scan image start
-------------------
Save window image (classified as ship) and gif image
scan time takes = 256.95370721817017 s
-------------------
---------------------------
NMS
---------------------------
Save NMS results


In [4]:
## Save data of images classified as ship

In [6]:
predict_ship2 = {'ship_pred_class': ship_pred_class, 'ship_img': ship_img, 'ship_cooridinate': ship_cooridinate}
    
with open('./results/capstoneP5/final_result/predict_ship_test2.pkl', 'wb') as file_name:
    pickle.dump(predict_ship2, file_name)

## 3) TEST IMAGE ON 2017-10-18 Long Beach

In [8]:
satimage_in = cv2.imread('./ships-in-satellite-imagery/test_v2/20171018.png')
satimage_out = cv2.imread('./ships-in-satellite-imagery/test_v2/20171018.png')
satimage_nms = cv2.imread('./ships-in-satellite-imagery/test_v2/20171018.png')

## Model 9 best model to predict another time

model = load_model('./cnn_model/classiferCNN/model9.h5')

(winH, winW) = (80, 80)
(stepY,stepX) = (10, 10)

dir_window = './results/capstoneP5/winimg1/'
dir_gif = './results/capstoneP5/gif1/'
dir_results = './results/capstoneP5/final_result/'

print('---------------------------')
print('Scan Sat Image')
print('---------------------------')

ship_pred_class, ship_img, ship_cooridinate = scan_image(satimage_in, satimage_out, model, dir_window, dir_gif, dir_results)

print('---------------------------')
print('NMS')
print('---------------------------')

picked_box = non_max_suppression(ship_cooridinate, area_overlap_threshold = 0.5)

for (bbx1, bby1, bbx2, bby2) in picked_box:
    cv2.rectangle(satimage_nms, (bbx1, bby1), (bbx2,bby2), (0, 0, 255), 2) 

        
cv2.imwrite("./results/capstoneP5/final_result/detectedtest3_NMS.png", satimage_nms)

print('Save NMS results')

---------------------------
Scan Sat Image
---------------------------
-------------------
Scan image start
-------------------
Save window image (classified as ship) and gif image
scan time takes = 141.901052236557 s
-------------------
---------------------------
NMS
---------------------------
Save NMS results


In [2]:
## Save data of images classified as ship

In [9]:
predict_ship3 = {'ship_pred_class': ship_pred_class, 'ship_img': ship_img, 'ship_cooridinate': ship_cooridinate}
    
with open('./results/capstoneP5/final_result/predict_ship_test3.pkl', 'wb') as file_name:
    pickle.dump(predict_ship3, file_name)

## 4) TEST IMAGE ON 2017-10-17 Long Beach

In [11]:
satimage_in = cv2.imread('./ships-in-satellite-imagery/test_v2/20171017.png')
satimage_out = cv2.imread('./ships-in-satellite-imagery/test_v2/20171017.png')
satimage_nms = cv2.imread('./ships-in-satellite-imagery/test_v2/20171017.png')

## Model 9 best model to predict another time

model = load_model('./cnn_model/classiferCNN/model9.h5')

(winH, winW) = (80, 80)
(stepY,stepX) = (10, 10)

dir_window = './results/capstoneP5/winimg1/'
dir_gif = './results/capstoneP5/gif1/'
dir_results = './results/capstoneP5/final_result/'

print('---------------------------')
print('Scan Sat Image')
print('---------------------------')

ship_pred_class, ship_img, ship_cooridinate = scan_image(satimage_in, satimage_out, model, dir_window, dir_gif, dir_results)

print('---------------------------')
print('NMS')
print('---------------------------')

picked_box = non_max_suppression(ship_cooridinate, area_overlap_threshold = 0.5)

for (bbx1, bby1, bbx2, bby2) in picked_box:
    cv2.rectangle(satimage_nms, (bbx1, bby1), (bbx2,bby2), (0, 0, 255), 2) 

        
cv2.imwrite("./results/capstoneP5/final_result/detectedtest4_NMS.png", satimage_nms)

print('Save NMS results')

---------------------------
Scan Sat Image
---------------------------
-------------------
Scan image start
-------------------
Save window image (classified as ship) and gif image
scan time takes = 291.79830527305603 s
-------------------
---------------------------
NMS
---------------------------
Save NMS results


In [3]:
## Save data of images classified as ship

In [13]:
predict_ship4 = {'ship_pred_class': ship_pred_class, 'ship_img': ship_img, 'ship_cooridinate': ship_cooridinate}
    
with open('./results/capstoneP5/final_result/predict_ship_test4.pkl', 'wb') as file_name:
    pickle.dump(predict_ship4, file_name)